## MEDI-TRACK-GPT

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Set the data path
data_dir = Path("../data/ehr")


# Load the Synthea CSV files
patients = pd.read_csv(data_dir / "patients.csv")
encounters = pd.read_csv(data_dir / "encounters.csv")
conditions = pd.read_csv(data_dir / "conditions.csv")
observations = pd.read_csv(data_dir / "observations.csv")

print("Loaded:", len(patients), "patients")
print("Loaded:", len(encounters), "encounters")

Loaded: 116 patients
Loaded: 7366 encounters


In [ ]:
# Reuse this function to simulate a doctor's discharge summary
def generate_discharge_note(encounter_row):
    encounter_id = encounter_row['Id']
    patient_id = encounter_row['PATIENT']
    reason = encounter_row['DESCRIPTION']

    # Get diagnoses
    diag_rows = conditions[conditions['ENCOUNTER'] == encounter_id]
    diagnoses = ", ".join(diag_rows['DESCRIPTION'].tolist()) or "No diagnosis available"

    # Get vitals
    vitals_rows = observations[observations['ENCOUNTER'] == encounter_id]
    vitals_text = ""
    for _, row in vitals_rows.iterrows():
        unit = row['UNIT'] if 'UNIT' in row else ''
        vitals_text += f"- {row['DESCRIPTION']}: {row['VALUE']} {unit}\n"


    note = f"""
    Discharge Summary
    ------------------
    Encounter ID: {encounter_id}
    Patient ID: {patient_id}
    Reason for Visit: {reason}
    Diagnoses: {diagnoses}

    Key Vitals:
    {vitals_text if vitals_text else 'No vitals recorded'}

    Plan:
    - Discharge with medications as prescribed.
    - Follow-up in 2 weeks.
    - If symptoms worsen, return to hospital immediately.
    """.strip()

    return note


In [ ]:
Path("data/ehr").mkdir(parents=True, exist_ok=True)

In [ ]:
# Create summaries for the first 50 encounters
summaries = []
for i, row in encounters.head(50).iterrows():
    summary = generate_discharge_note(row)
    summaries.append({'encounter_id': row['Id'], 'summary': summary})

# Save to CSV for GPT input
summary_df = pd.DataFrame(summaries)
summary_df.to_csv("data/ehr/discharge_summaries.csv", index=False)
summary_df.head(3)


,encounter_id,summary
0,40f237cc-477c-6442-3e33-7e872da13ba7,Discharge Summary\n ------------------\n ...
1,f21d1bba-2a97-4b5d-6063-06dc8d8c610e,Discharge Summary\n ------------------\n ...
2,7b2e8293-5681-eea9-5e10-81b11b5d86cc,Discharge Summary\n ------------------\n ...
